(sec:exercise-othello)=
# 演習3 - オセロエージェントを作る

今回の演習では、前節、[基本のオセロAI](sec:othello-agent)の延長として、より強いオセロのAIを作成してみよう。

前節では、オセロAIの基本であるゲーム木の探索方法について、マス評価値に基づいた[ミニマックス探索](ssec:minimax)と、マス評価値に依らない[原始モンテカルロ探索](ssec:monte-carlo)について解説した。

本演習では、これらの延長として、Q関数の**関数近似**に基づいたAIの強化に取り組んでみよう。

In [1]:
import os
import random
import warnings

import numpy as np
import seaborn as sns
import matplotlib
from sklearn.exceptions import ConvergenceWarning

# グラフの設定
matplotlib.rcParams["figure.dpi"] = 150
sns.set(style="white", palette="colorblind")

# シードの固定
random.seed(31415)
np.random.seed(31415)

# 一部の警告を無視
warnings.simplefilter("ignore", ConvergenceWarning)
warnings.simplefilter("ignore", FutureWarning)
os.environ["PYTHONWARNINGS"] = "ignore"

In [2]:
try:
    from myst_nb import glue
except ImportError:
    glue = lambda *args, **kwargs: _

In [3]:
# joblib用のtqdm
import contextlib

import joblib
from tqdm.notebook import tqdm


@contextlib.contextmanager
def tqdm_joblib(total=None, **kwargs):
    pbar = tqdm(total=total, miniters=1, smoothing=0, **kwargs)

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            pbar.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback

    try:
        yield pbar
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        pbar.close()

## Q学習における関数近似

**関数近似**とは、Q関数$Q(s, a)$を何らかの機械学習モデルによって近似する手法を指す。[Q学習の基本](sec:q-learning)で紹介した手法ではQテーブルを離散化していたが、このような手法は、状態数の多いゲームでは適用が難しい。

今回、演習で扱うオセロも取り得る盤の状態は、64個のマスについて、黒、白、空の3通りが考えられるから、その状態数は$3^{64} \approx 3 \times 10^{30}$にもなる。従って、このような大量の状態数に対して、そのまま離散的なQテーブルを定義することは現実的ではない。

そこで、状態$s$を入力とし、取り得る行動$a \in \mathcal{A}$について、行動価値を与えるような関数$Q(s, a)$を機械学習モデルによって表現することを考える。

最も単純な例として、状態を表わすパラメータをベクトル$\mathbf{s}$として表わし、この線形変換により、各行動の価値を要素に持つベクトル$\mathbf{a}$を求めるようなモデルを考えることができる。

$$
\mathbf{a} = \mathbf{W} \mathbf{s} + \mathbf{b}
$$

このような線形モデルであれば、Q学習の更新式:

$$
Q_{\rm new}(s, a) = Q(s,a ) + \alpha \left[ R(s, a) + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]
$$

の第2項に現れるTD誤差を最小化するようにモデルのパラメータ$\mathbf{W}$, $\mathbf{b}$を最適化すれば良い。

### scikit-learnを用いた関数近似

In [4]:
import othello
from othello import Move, Player

env = othello.make()

In [5]:
from sklearn.neural_network import MLPRegressor

reg = MLPRegressor(
    hidden_layer_sizes=(128,),
    activation="relu",
    learning_rate="constant",
    learning_rate_init=1.0e-4,
    alpha=0.0,
    batch_size=32,
    shuffle=True,
    warm_start=True,
)

dummy_X = np.zeros((reg.batch_size, 8 * 8 * 2))
dummy_y = np.zeros((reg.batch_size, 8 * 8 + 1))
reg.partial_fit(dummy_X, dummy_y)

MLPRegressor(alpha=0.0, batch_size=32, hidden_layer_sizes=(128,),
             learning_rate_init=0.0001, warm_start=True)

In [6]:
def get_move_index(move):
    if move.is_pass():
        return 64
    return move.y * 8 + move.x

In [7]:
def get_feature(player, board):
    b0 = np.maximum(0, +1.0 * player * board)
    b1 = np.maximum(0, -1.0 * player * board)
    feature = np.concatenate([b0.flatten(), b1.flatten()])
    return feature

In [8]:
def move_by_random(env, player):
    """有効手の中からランダムに手を選ぶ"""
    moves = env.legal_moves(player)
    if len(moves) == 0:
        return Move.Pass(player)
    else:
        return np.random.choice(moves)

In [9]:
def match():
    # ゲームのリセット
    env = othello.make()
    player, board = env.reset()

    # エピソード開始
    while not env.gameset():
        # 黒番:
        if player == Player.BLACK:
            # セル評価値が最も高い場所に着手する
            moves = env.legal_moves(player)

            if len(moves) == 0:
                move = Move.Pass(player)
            else:
                best_move = None
                best_score = -np.inf
                feature = get_feature(player, board)
                action_values = reg.predict([feature])[0]
                for m in moves:
                    value = action_values[get_move_index(m)]
                    if best_score < value:
                        best_score = value
                        best_move = m

                move = best_move
        # 白番:
        if player == Player.WHITE:
            # 着手可能な手があればランダムに1つを選ぶ
            move = move_by_random(env, player)

        # 着手による盤の状態の更新
        player, board = env.step(move)

    return env.count()

In [10]:
def win_score(n_matches=1000, n_jobs=4, verbose=False):
    if verbose:
        with tqdm_joblib(n_matches):
            result = joblib.Parallel(n_jobs=n_jobs)(
                (joblib.delayed(match)() for _ in range(n_matches)),
            )
    else:
        result = joblib.Parallel(n_jobs=n_jobs)(
            (joblib.delayed(match)() for _ in range(n_matches)),
        )
        
    result = np.array(result, dtype="int32")
    b_win = np.sum(result[:, 0] > result[:, 1])
    w_win = np.sum(result[:, 0] < result[:, 1])
    draw = np.sum(result[:, 0] == result[:, 1])

    return b_win / n_matches

In [11]:
n_episodes = 10
n_matches = 100
n_epochs = 100
gamma = 0.999

e0 = 0.5
e1 = 0.1
e_scale = np.exp((np.log(e1) - np.log(e0)) / (n_episodes - 1))
epsilon = e0

pbar = tqdm(total=n_episodes * n_epochs)
win = 0.0
for _ in range(n_episodes):
    X = []
    y = []
    for _ in range(n_matches):
        player, board = env.reset()
        while not env.gameset():
            # 入力のベクトルは現在の状態
            feature = get_feature(player, board)
            action_values = reg.predict([feature])[0]
            X.append(feature.copy())

            # 手を指して状態を進める
            # 以下はε-greedy法で手を選ぶ
            moves = env.legal_moves(player)
            if len(moves) == 0:
                move = Move.Pass(player)
            elif np.random.uniform(0.0, 1.0) < epsilon:
                move = np.random.choice(moves)
            else:
                best_move = None
                best_score = -np.inf
                for m in moves:
                    value = action_values[get_move_index(m)]
                    if best_score < value:
                        best_score = value
                        best_move = m
                move = best_move

            next_player, next_board = env.step(move)

            reward = 0.0
            if env.gameset():
                # 対戦結果に応じて報酬(罰)を与える
                n_now = env.count(player)
                n_next = env.count(next_player)
                if n_now > n_next:
                    reward = 1.0
                elif n_now < n_next:
                    reward = -1.0
                else:
                    reward = 0.0

                # 次の局面はないので, 次局面からの報酬伝搬はない
                next_best_score = 0.0
            else:
                # 相手にとって最も良いQ値を得る
                next_moves = env.legal_moves(next_player)
                feature = get_feature(next_player, next_board)
                next_action_values = reg.predict([feature])[0]

                if len(next_moves) == 0:
                    next_best_score = next_action_values[-1]
                else:
                    next_best_score = -np.inf
                    for m in next_moves:
                        value = next_action_values[get_move_index(m)]
                        if next_best_score < value:
                            next_best_score = value

                # 相手の報酬は自分にとっては損なのでマイナスにする
                next_best_score *= -1.0
                
            # 着手に対応する行動価値を更新
            action_index = get_move_index(move)
            target = action_values
            target[action_index] = reward + gamma * next_best_score
            y.append(target.copy())

            # 変数を更新
            player = next_player
            board = next_board

    # 1エピソードごとに回帰モデルを更新
    X = np.stack(X, axis=0)
    y = np.stack(y, axis=0)
    for epoch in range(n_epochs):
        reg = reg.partial_fit(X, y)
        pbar.set_description(f"win={win:.3f}, loss={reg.loss_:.4f}")
        pbar.update()

    win = win_score()
    epsilon *= e_scale

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
glue("ql_b_win", b_win, display=True)
glue("ql_w_win", w_win, display=True)
glue("ql_draw", draw, display=True)

NameError: name 'b_win' is not defined

**対局結果: Q学習 vs ランダム**
- 黒番勝ち: {glue:}`ql_b_win`
- 白番勝ち: {glue:}`ql_w_win`
- 引き分け: {glue:}`ql_draw`

## 演習内容

今回の演習では以下のようなオセロをプレイするエージェントのクラスを実装する。ただし、基本的にはオセロ環境が渡されてくる`Agent.play`だけを編集すれば良い。

In [ ]:
import numpy as np


class Agent(object):
    def __init__(self):
        pass

    def play(self, player, env) -> Move:
        """
        この関数を更新、以下はランダムに着手する例
        """
        moves = env.legal_moves()
        if len(moves) != 0:
            return Move.Pass(player)
        else:
            return np.random.choice(moves)

### 対戦相手のレベル

- **レベル1:** セル評価値を用いたアルファベータ探索で2手先まで読むAI
- **レベル2:** 上記のscikit-learnを用いた関数近似によって、次の手の価値を評価するAI
- **レベル3:** Q関数の学習方法を改良し、さらにQ関数に基づいて2手先まで読むAI

### ローカル環境でのテスト

### 本番環境でのテスト